In [5]:
import os
import glob
import xml.etree.ElementTree as ET
import re
import pandas as pd
from collections import Counter

In [6]:
directory = 'Project3_data'
xml_files = glob.glob(os.path.join(directory, '*.xml'))
vital_signs_pattern = {}
smoker_df = pd.DataFrame(columns=['patient_number', 'smoker_status'])


files_with_vital_signs = []
for xml_file in xml_files:
    ## file_number = os.path.basename(xml_file).replace('.xml', '')
    patient_number = re.search(r'(\d+)-', os.path.basename(xml_file)).group(1)
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for smoker in root.findall('.//SMOKER'):
        smoker_status = smoker.get('status')
        row_df = pd.DataFrame([{'patient_number': patient_number, 'smoker_status': smoker_status}])
        smoker_df = pd.concat([smoker_df, row_df], ignore_index=True)

smoker_df.to_csv('smoker.csv', index=False)

In [7]:
smoker_df = smoker_df.drop_duplicates(subset=['patient_number', 'smoker_status'])
len(smoker_df)
## sort by patient_number and create csv
smoker_df = smoker_df.sort_values(by='patient_number')
smoker_df['smoker_status'].unique()

array(['never', 'unknown', 'current', 'past', 'ever'], dtype=object)

In [8]:
# Define the hierarchy
# ['past', 'current', 'unknown', 'never', 'smoker_status']
status_hierarchy = {'current': 1, 'past': 2, 'ever': 3, 'never': 4, 'unknown': 5}

i = 0
while i < len(smoker_df):
  cur_patient_number = smoker_df.iloc[i]['patient_number']
  status_array = []
  start_index = i
  while i < len(smoker_df) and smoker_df.iloc[i]['patient_number'] == cur_patient_number:
      status_array.append(smoker_df.iloc[i]['smoker_status'])
      i += 1
  status_array.sort(key=lambda x: status_hierarchy[x])
  if status_array[0] == 'ever':
      print('hello')
  smoker_df.loc[smoker_df['patient_number'] == cur_patient_number, 'smoker_status'] = status_array[0]

smoker_df = smoker_df.drop_duplicates()
print(smoker_df['smoker_status'].unique())
smoker_df.to_csv('smoker.csv', index=False)

['current' 'unknown' 'never' 'past']
